## 2. LIMPIEZA

### 1. Importamos las librerías necesarias.

In [1]:
# Herramientas para EDA
import pandas as pd
import numpy as np
import sidetable as stb
from datetime import timedelta
from datetime import date, datetime

# Herramientas para la Visualización
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import colors 

### 2. Abrimos el archivo con el que vamos a trabajar.

In [2]:
df = pd.read_csv('../archivos/bikes.csv', index_col=0)
df.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,spring,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,spring,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,spring,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


### 3. Después de un EDA inicial, procedemos a la limpieza de nuestros datos.

3.1. Eliminación de columnas.

In [3]:
# Eliminamos la columna instant, porqué es redundante

df.drop(['instant'], axis=1, inplace=True)

3.2. Traducción de columnas y valores.

In [4]:
# Renombramos las columnas para que esté toda nuestra info en un mismo idioma
nuevas_columnas= ['fecha', 'estacion', 'año', 'mes', 'dias_festivos', 'dias_de_la_semana', 'dias_laborales', 'clima', 'temperatura', 'sensacion_termica', 'humedad', 'velocidad_viento', 'clientes_casuales', 'clientes_registrados', 'total_bicis']
df.columns = nuevas_columnas

3.3. Corregimos los días de la semana.

In [5]:
for i,v in enumerate(df['fecha']):
    days= ['1', '2', '3', '4', '5', '6', '7']
    d= datetime.strptime(v, '%d-%m-%Y') #damos la vuelta a la fecha
    df.loc[i,'dia_semana'] = days[d.weekday()] 

In [6]:
df.head()

,fecha,estacion,año,mes,dias_festivos,dias_de_la_semana,dias_laborales,clima,temperatura,sensacion_termica,humedad,velocidad_viento,clientes_casuales,clientes_registrados,total_bicis,dia_semana
0,01-01-2018,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,1
1,02-01-2018,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801,2
2,03-01-2018,spring,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349,3
3,04-01-2018,spring,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562,4
4,05-01-2018,spring,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600,5


In [7]:
df.drop(['dias_de_la_semana'], axis=1, inplace=True)

3.4. Las estaciones estaban intercambiadas, ponemos los nombres correctos.

In [8]:
df['estacion'].unique()

array(['spring', 'summer', 'autumn', 'winter'], dtype=object)

In [9]:
dic_est = {'spring' : 'invierno', 'winter' : 'otoño', 'summer' : 'primavera', 'autumn' : 'verano'}
df['estacion'] = df['estacion'].map(dic_est)

In [10]:
df.sample(5)

,fecha,estacion,año,mes,dias_festivos,dias_laborales,clima,temperatura,sensacion_termica,humedad,velocidad_viento,clientes_casuales,clientes_registrados,total_bicis,dia_semana
89,31-03-2018,primavera,0,3,0,1,3,11.001653,12.87875,91.8333,14.582282,179,1506,1685,6
284,12-10-2018,otoño,0,10,0,1,3,22.276653,25.88585,90.6250,16.626050,217,2199,2416,5
667,30-10-2019,otoño,1,10,0,1,2,13.045462,15.49545,82.5455,14.271603,87,1009,1096,3
682,14-11-2019,otoño,1,11,0,1,1,11.855847,14.07815,55.2083,13.374875,373,5122,5495,4
348,15-12-2018,otoño,0,12,0,1,2,17.322500,20.61185,63.4167,17.958814,181,3528,3709,6


3.5. Corregimos los días festivos.

In [11]:
df['dias_festivos'].value_counts()

0    709
1     21
Name: dias_festivos, dtype: int64

In [12]:
df['fecha'] = pd.to_datetime(df['fecha'], format="%d-%m-%Y")

In [13]:
#!pip install holidays
import holidays

In [14]:
df['dias_festivos'] = pd.Series(df['fecha']).apply(lambda x: holidays.CountryHoliday('US', state='WA').get(x)).values

In [15]:
df['dias_festivos'].nunique()

11

In [16]:
df['dias_festivos'].value_counts()

New Year's Day                2
Martin Luther King Jr. Day    2
Washington's Birthday         2
Memorial Day                  2
Independence Day              2
Labor Day                     2
Columbus Day                  2
Veterans Day                  2
Thanksgiving                  2
Christmas Day                 2
Veterans Day (Observed)       1
Name: dias_festivos, dtype: int64

In [17]:
df['dias_festivos_cat'] = df['dias_festivos'].astype('bool').astype('int')

In [18]:
df[(df['dias_festivos'] == 'Veterans Day') | (df['dias_festivos'] == 'Veterans Day (Observed)')] 
#en 2018 pasan el veterands day al lunes, por lo que tenemos un día más festivo (un día menos laborable)

,fecha,estacion,año,mes,dias_festivos,dias_laborales,clima,temperatura,sensacion_termica,humedad,velocidad_viento,clientes_casuales,clientes_registrados,total_bicis,dia_semana,dias_festivos_cat
314,2018-11-11,otoño,0,11,Veterans Day,0,1,13.290847,15.34085,44.6250,21.083225,440,2928,3368,7,1
315,2018-11-12,otoño,0,11,Veterans Day (Observed),0,1,14.623347,17.89710,55.2917,14.208154,1275,2792,4067,1,1
679,2019-11-11,otoño,1,11,Veterans Day,0,1,17.254153,21.08565,65.9167,8.542500,2290,4562,6852,1,1


In [19]:
df.head(2)

,fecha,estacion,año,mes,dias_festivos,dias_laborales,clima,temperatura,sensacion_termica,humedad,velocidad_viento,clientes_casuales,clientes_registrados,total_bicis,dia_semana,dias_festivos_cat
0,2018-01-01,invierno,0,1,New Year's Day,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,1,1
1,2018-01-02,invierno,0,1,None,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801,2,0


---

In [20]:
df.to_csv('../archivos/bikes_limpio.csv')

3.6. Estandarizamos los outliers.

In [21]:
# Ahora nos centramos en nuestros outliers
numericas = df.select_dtypes(include=np.number) 
numericas.head()

,año,mes,dias_laborales,clima,temperatura,sensacion_termica,humedad,velocidad_viento,clientes_casuales,clientes_registrados,total_bicis,dias_festivos_cat
0,0,1,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985,1
1,0,1,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801,0
2,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349,0
3,0,1,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562,0
4,0,1,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600,0


In [22]:
lista_columnas = numericas.columns  

In [23]:
def detectar_outliers(lista_columnas, dataframe): 
    
    dicc_indices = {} 
    
    for col in lista_columnas:
        
        Q1 = np.nanpercentile(numericas[col], 25)
        Q3 = np.nanpercentile(numericas[col], 75)
     
        IQR = Q3 - Q1
        
        outlier_step = 1.5 * IQR
    
        outliers_data = dataframe[(dataframe[col] < Q1 - outlier_step) | (dataframe[col] > Q3 + outlier_step)]
        
        
        if outliers_data.shape[0] > 0: 
        
            dicc_indices[col] = (list(outliers_data.index)) 
        
    return dicc_indices 

In [24]:
indices= detectar_outliers(lista_columnas, df)  

In [25]:
indices.keys() 

dict_keys(['humedad', 'velocidad_viento', 'clientes_casuales', 'dias_festivos_cat'])

In [26]:
outliers_humedad = list(indices['humedad'])
outliers_velocidad_viento = list(indices['velocidad_viento'])
outliers_festivos = list(indices['dias_festivos'])
outliers_clientes = list(indices['clientes_casuales'])

KeyError: 'dias_festivos'

In [ ]:
#generamos un for loop para reemplazar los valores atípicos de 'humedad':
for i in outliers_humedad:
    df.replace(numericas.loc[i,'humedad'], numericas['humedad'].median(), inplace=True)

In [ ]:
#generamos un for loop para reemplazar los valores atípicos de 'velocidad_viento':
for i in outliers_velocidad_viento:
    df.replace(numericas.loc[i,'velocidad_viento'], numericas['velocidad_viento'].median(), inplace=True)

In [ ]:
#generamos un for loop para reemplazar los valores atípicos de 'dias festivos':
for i in outliers_festivos:
    df.replace(numericas.loc[i,'dias_festivos'], numericas['dias_festivos'].median(), inplace=True)

In [ ]:
#generamos un for loop para reemplazar los valores atípicos de 'clientes_casuales':
for i in outliers_clientes:
    df.replace(numericas.loc[i,'clientes_casuales'], numericas['clientes_casuales'].median(), inplace=True)